# Test creating a DEM using the orthoimages

In [ ]:
import os
from glob import glob
import subprocess
from tqdm import tqdm
import numpy as np
import rioxarray as rxr
import xarray as xr
import matplotlib.pyplot as plt

# Define paths to orthoimages, cameras, and reference DEM
data_folder = '/Users/rdcrlrka/Research/Soo_locks'
out_folder = os.path.join(data_folder, '20251001_imagery', 'frames_IR_proc_out')
ortho_folder = os.path.join(out_folder, 'final_ortho')
cam_folder = os.path.join(out_folder, 'bundle_adjust')
refdem_file = os.path.join(os.getcwd(), '..', 'inputs', '20251001_Soo_Model_1cm_mean_UTM19N-fake_filled.tif')

# Get images and cameras
image_list = sorted(glob(os.path.join(ortho_folder, '*.tiff')))
cam_list = sorted(glob(os.path.join(cam_folder, '*.tsai')))
print(f'Located {len(image_list)} images and {len(cam_list)} cameras')

# Define output folder
final_stereo_folder = os.path.join(out_folder, 'final_stereo')
os.makedirs(final_stereo_folder, exist_ok=True)

## Run stereo

In [ ]:
# Set up image pairs
image1_list, image2_list = image_list[0:-1], image_list[1:]
cam1_list, cam2_list = cam_list[0:-1], cam_list[1:]

# Iterate over pairs
for i in tqdm(range(len(image1_list))):
    image1, image2 = image1_list[i], image2_list[i]
    cam1, cam2 = cam1_list[i], cam2_list[i]

    pair_prefix = os.path.join(
        final_stereo_folder,
        os.path.splitext(os.path.basename(image1))[0] + '__' + os.path.splitext(os.path.basename(image2))[0],
        'run'
        )
    
    cmd = [
        'parallel_stereo',
        '--threads-singleprocess', '12',
        '--threads-multiprocess', '12',
        image1, image2,
        cam1, cam2,
        pair_prefix,
        refdem_file
    ]
    subprocess.run(cmd)

## Rasterize point clouds

In [ ]:
pc_files = sorted(glob(os.path.join(final_stereo_folder, '*', '*-PC.tif')))
for pc in tqdm(pc_files):
    cmd = [
        'point2dem',
        '--threads', '12',
        '--tr', '0.01',
        pc
    ]
    subprocess.run(cmd)

## Mosaic DEMs

In [ ]:
dem_fns = sorted(glob(os.path.join(final_stereo_folder, '*', '*DEM.tif')))
mosaic_fn = os.path.join(final_stereo_folder, f'DEM_mosaic.tif')
cmd = [
    'dem_mosaic',
    '--threads', '12',
    '-o', mosaic_fn
] + dem_fns
subprocess.run(cmd)

## Plot results

In [ ]:
# Load the input files
def load_raster(raster_file):
    raster = rxr.open_rasterio(raster_file).squeeze()
    crs = raster.rio.crs
    raster = xr.where(raster < -100, np.nan, raster)
    raster.rio.write_crs(crs, inplace=True)
    return raster
ortho_file = os.path.join(ortho_folder, 'orthomosaic.tif')
ortho = load_raster(ortho_file)
dem_file = os.path.join(final_stereo_folder, 'DEM_mosaic.tif')
dem = load_raster(dem_file)
refdem = load_raster(refdem_file)
refdem = refdem.rio.reproject_match(dem)

plt.rcParams.update({'font.sans-serif': 'Verdana', 'font.size': 12})
fig, ax = plt.subplots(1, 3, figsize=(18,8))
# Ortho
ax[0].imshow(
    ortho,
    cmap='Grays_r',
    extent=(min(ortho.x), max(ortho.x), min(ortho.y), max(ortho.y))
)
ax[0].set_title('IR image mosaic')
# DEM
im = ax[1].imshow(
    dem, 
    cmap='terrain', 
    extent=(min(dem.x), max(dem.x), min(dem.y), max(dem.y), 'meters')
    )
cb = fig.colorbar(im, shrink=0.5)
ax[1].set_title('DSM mosaic')
# DEM - refdem
im = ax[2].imshow(
    dem - refdem, 
    cmap='coolwarm_r',
    clim=(-1,1),
    extent=(min(dem.x), max(dem.x), min(dem.y), max(dem.y))
    )
cb = fig.colorbar(im, shrink=0.5, label='meters')
ax[2].set_title('DSM mosaic - Lidar mean')

for axis in ax:
    axis.set_xticks([])
    axis.set_yticks([])

fig.tight_layout()
plt.show()

# Save to file
fig_file = os.path.join(out_folder, 'result.jpg')
fig.savefig(fig_file, dpi=300, bbox_inches='tight')
print('Figure saved to file:', fig_file)